# Residual Networks (ResNet)
Very deep networks (more layers) often result in gradients that vanishes as the gradient is back-propagated to earlier layers, repeated multiplication may make the gradient infinitely small.<br>
i.e when we increase the number of layers, there is a common problem in deep learning called the Vanishing/Exploding gradient. This causes the gradient to become 0 or too large. Thus when we increases number of layers, the training and test error rate also increases. <br>In order to solve the problem of the vanishing/exploding gradient, this architecture introduced the concept called Residual Blocks. In this network, we use a technique called skip connections. The skip connection connects activations of a  layer to further layers by skipping some layers in between. This forms a residual block. Resnets are made by stacking these residual blocks together. The approach behind this network is instead of layers learning the underlying mapping, we allow the network to fit the residual mapping.

So, instead of say H(x), initial mapping, let the network fit,

F(x) := H(x) - x which gives H(x) := F(x) + x.

![image](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*3wubYL0XmePeU1KH7cMAXQ.png)

advantage of adding this type of skip connection is that if any layer hurt the performance of architecture then it will be skipped by regularization.


![image.png](https://media.licdn.com/dms/image/v2/D4E12AQHOV3z8WjgjjA/article-cover_image-shrink_720_1280/article-cover_image-shrink_720_1280/0/1670533041839?e=1739404800&v=beta&t=nFYe9G14YWkEOfkXVXlBBCtNWnrpfMLy6whYhdOOH1s)

It starts with a convolution layer of 7x7 sized kernel(64) with a stride of 2 followed by a MaxPooling operation.
It consists of four residual blocks (config:- 3,4,6 and 3 respectively)
Channels for each block are constant— 64, 128, 256, 512 respectively.
Only 3x3 kernels have been used in these blocks.
Except for the first block, each block starts with a 3x3 kernel of stride of 2.

In [ ]:
from tensorflow.keras import layers,models

In [ ]:
def residual_block(input_x,num_filters,downsample=False):

    strides=2 if downsample else 1

    x=layers.Conv2D(num_filters,(3,3),padding='same',strides=strides,activation='relu')(input_x)
    x=layers.Conv2D(num_filters,(3,3),padding='same',strides=1)(x)

    skip_x=input_x

    if downsample or input_x.shape[-1]!=num_filters:

        skip_x=layers.Conv2D(num_filters,(1,1),strides=strides)(skip_x)

    x=layers.Add()([x,skip_x])

    return layers.Activation('relu')(x)



In [ ]:
def build_resnet(input_shape=(224,224,3),num_classes=1000):

    inputs=layers.Input(input_shape)

    # 7x7 conv , 64 filters , with stride =2

    x=layers.Conv2D(64,(7,7),strides=2,padding='same',activation='relu')(inputs)

    # 3x3 pooling with stride =2

    x=layers.MaxPooling2D((3,3),strides=2,padding='same')(x)

    x=residual_block(x,64) #residual block 1

    x=residual_block(x,128,downsample=True)

    x=residual_block(x,256,downsample=True)

    x=residual_block(x,512,downsample=True)

    x=layers.GlobalAveragePooling2D()(x)

    outputs=layers.Dense(num_classes,activation='softmax')(x)

    return models.Model(inputs,outputs)


In [ ]:
resnet_model=build_resnet()

In [ ]:
resnet_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 112, 112, 64)   │          9,472 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 56, 56, 64)     │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 56, 56, 64)     │         36,928 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 56, 56, 64)     │         36,928 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 56, 56, 64)     │              0 │ conv2d_5[0][0],        │
│                           │                        │                │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 56, 56, 64)     │              0 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 28, 28, 128)    │         73,856 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 28, 28, 128)    │        147,584 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 28, 28, 128)    │          8,320 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 28, 28, 128)    │              0 │ conv2d_7[0][0],        │
│                           │                        │                │ conv2d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 28, 28, 128)    │              0 │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 14, 14, 256)    │        295,168 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_10 (Conv2D)        │ (None, 14, 14, 256)    │        590,080 │ conv2d_9[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_11 (Conv2D)        │ (None, 14, 14, 256)    │         33,024 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 14, 14, 256)    │              0 │ conv2d_10[0][0],       │
│                           │                        │                │ conv2d_11[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 14, 14, 256)    │              0 │ add_2[0][0]            │
├──────────────────────

 Total params: 5,415,912 (20.66 MB)

 Trainable params: 5,415,912 (20.66 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
!wget https://www.dropbox.com/s/3o35wiiwua3qtp8/chest_xray.zip

--2024-12-12 16:18:39--  https://www.dropbox.com/s/3o35wiiwua3qtp8/chest_xray.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/npqab6p2dj0yf9bcko8ub/chest_xray.zip?rlkey=5jxsmpq7yo56597d6vjhjafql [following]
--2024-12-12 16:18:40--  https://www.dropbox.com/scl/fi/npqab6p2dj0yf9bcko8ub/chest_xray.zip?rlkey=5jxsmpq7yo56597d6vjhjafql
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5ac4887866a873e225c77c2309.dl.dropboxusercontent.com/cd/0/inline/CgFQhVpBLxSgBOnkxrHpd8vg_LvQgAw3JZWrkyTjwg3Dpsd96W51eaT0shqhEyaH10P3YyvW6n7ISK4wpl8tCzG4afUD8rje8w3g2Tny6v8T3CEAduScn1m7oNeFkJLoX5j8BZpVKdp_vorzCritwOW6/file# [following]
--2024-12-12 16:18:40--  https://uc5ac4887866a873e225c77c2309.dl.dropboxusercontent.com/cd/0/in

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image

In [ ]:
base_model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
base_model.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
base_model.output

<KerasTensor shape=(None, 7, 7, 2048), dtype=float32, sparse=False, name=keras_tensor_174>

In [ ]:
#from tensorflow.keras.regularizers import L1

In [ ]:
x = base_model.output
global_pool =GlobalAveragePooling2D()(x)
dense1 = Dense(256,activation='relu')(global_pool)
batch_norm =BatchNormalization()(dense1)
dropout = Dropout(0.3)(batch_norm)
output = Dense(2,activation='softmax')(dropout)

model = Model(inputs=base_model.input,outputs=output)

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 24,113,794 (91.99 MB)

 Trainable params: 24,060,162 (91.78 MB)

 Non-trainable params: 53,632 (209.50 KB)

In [ ]:
# Freeze the base MobileNet layers so they won't be trained
for layer in base_model.layers:
  layer.trainable=False

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 24,113,794 (91.99 MB)

 Trainable params: 525,570 (2.00 MB)

 Non-trainable params: 23,588,224 (89.98 MB)

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
import os

In [ ]:
!unzip chest_xray.zip

Streaming output truncated to the last 5000 lines.
  inflating: chest_xray/train/PNEUMONIA/person67_bacteria_334.jpeg  
  inflating: __MACOSX/chest_xray/train/PNEUMONIA/._person67_bacteria_334.jpeg  
  inflating: chest_xray/train/PNEUMONIA/person480_bacteria_2039.jpeg  
  inflating: __MACOSX/chest_xray/train/PNEUMONIA/._person480_bacteria_2039.jpeg  
  inflating: chest_xray/train/PNEUMONIA/person40_bacteria_202.jpeg  
  inflating: __MACOSX/chest_xray/train/PNEUMONIA/._person40_bacteria_202.jpeg  
  inflating: chest_xray/train/PNEUMONIA/person413_bacteria_1829.jpeg  
  inflating: __MACOSX/chest_xray/train/PNEUMONIA/._person413_bacteria_1829.jpeg  
  inflating: chest_xray/train/PNEUMONIA/person1502_virus_2612.jpeg  
  inflating: __MACOSX/chest_xray/train/PNEUMONIA/._person1502_virus_2612.jpeg  
  inflating: chest_xray/train/PNEUMONIA/person1400_bacteria_3554.jpeg  
  inflating: __MACOSX/chest_xray/train/PNEUMONIA/._person1400_bacteria_3554.jpeg  
  inflating: chest_xray/train/PNEUMONIA/p

In [ ]:
os.listdir('/content/chest_xray')

['train', '.DS_Store', 'test', 'val']

In [ ]:
!rm -rf /content/chest_xray/.DS_Store

In [ ]:
os.listdir('/content/chest_xray')

['train', 'test', 'val']

In [ ]:
train_dir = '/content/chest_xray/train'
test_dir = '/content/chest_xray/test'
val_dir = '/content/chest_xray/val'

In [ ]:
train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                        rescale=1./255,
    rotation_range=10,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift                  # data augmentation
    shear_range=0.2,  # Random shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest')  # Fill in any pixels left after transformations )

val_data_gen =ImageDataGenerator(preprocessing_function=preprocess_input,
                                  rescale=1./255,
                                 rotation_range=10,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Random shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest')  # Fill in any pixels left after transformations )



test_data_gen = ImageDataGenerator(preprocess_input, rescale=1./255 )



In [ ]:
train_generator = train_data_gen.flow_from_directory(train_dir,
                                                    target_size=(224,224),
                                                    batch_size=32,
                                                    class_mode='categorical')

test_generator = test_data_gen.flow_from_directory(test_dir,
                                                    target_size=(224,224),
                                                    batch_size=32,
                                                    class_mode='categorical')

val_generator = val_data_gen.flow_from_directory(val_dir,
                                                    target_size=(224,224),
                                                    batch_size=32,
                                                    class_mode='categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [ ]:
train_generator.n // train_generator.batch_size

163

In [ ]:
train_generator.n

5216

In [ ]:
train_generator.batch_size

32

In [ ]:
len(train_generator), len(val_generator)

(163, 1)

tried epochs, early stopping but my val_accuracy was 56 followed by 65 again and again, it was performing good on training data sometimes even 100

## tried
Data augmentation is one of the best ways to combat overfitting, as it artificially increases the diversity of your training dataset by applying random transformations to the images.<br> increased dropout ->then accuray dropped and added batch_normalization.




can try  Regularization techniques such as L2 weight decay (Ridge Regularization) or Dropout can help reduce overfitting by penalizing overly large weights or randomly setting some neurons' outputs to zero during training.

Learning Rate Scheduling:<br>
Sometimes, reducing the learning rate as the training progresses helps in preventing overfitting. Use learning rate schedules like ReduceLROnPlateau or custom schedules: This will reduce the learning rate by a factor of 0.5 if the validation loss does not improve for 3 epochs.

In [ ]:
# from keras.callbacks import ReduceLROnPlateau
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
#                               patience=3, min_lr=1e-6)

in model.fit() we only pass x not y bcoz when data is stored as generator, the generator itself actually contains the corresponding labels. so we donot need to specify them seperately.

In [ ]:
model.fit(train_generator,
          epochs=5,
          validation_data=val_generator,
          steps_per_epoch=160,
          validation_steps=160,
          #callbacks =[reduce_lr]
          )

Epoch 1/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 103s 619ms/step - accuracy: 0.8673 - loss: 0.3294 - val_accuracy: 0.5000 - val_loss: 1.3110
Epoch 2/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8436 - loss: 0.3113 - val_accuracy: 0.5000 - val_loss: 1.1731
Epoch 3/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 103s 613ms/step - accuracy: 0.8737 - loss: 0.2970 - val_accuracy: 0.5000 - val_loss: 1.7969
Epoch 4/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8540 - loss: 0.3453 - val_accuracy: 0.5000 - val_loss: 1.6690
Epoch 5/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 141s 620ms/step - accuracy: 0.8723 - loss: 0.2962 - val_accuracy: 0.6250 - val_loss: 0.8188


In [ ]:
myimg ='/content/chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg'

In [ ]:
def process_image_for_testing(my_img):
  img = image.load_img(my_img, target_size=(224, 224))
  img_tensor = image.img_to_array(img)
  img_tensor = np.expand_dims(img_tensor, axis=0)
  img_tensor /= 255.
  return img_tensor

In [ ]:
model.predict(process_image_for_testing(myimg))

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


array([[0.9362133 , 0.06378672]], dtype=float32)

In [ ]:
img2 ='/content/chest_xray/test/PNEUMONIA/person100_bacteria_479.jpeg'

In [ ]:
model.predict(process_image_for_testing(img2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


array([[0.65836066, 0.34163934]], dtype=float32)

In [ ]:
img3= '/content/chest_xray/test/NORMAL/IM-0009-0001.jpeg'

In [ ]:
model.predict(process_image_for_testing(img3))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


array([[0.9410657 , 0.05893427]], dtype=float32)

In [ ]:
img4 ='/content/chest_xray/test/NORMAL/IM-0010-0001.jpeg'

In [ ]:
model.predict(process_image_for_testing(img4))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


array([[0.87896997, 0.12103002]], dtype=float32)

In [ ]:
img5 ='/content/chest_xray/test/PNEUMONIA/person104_bacteria_491.jpeg'

In [ ]:
model.predict(process_image_for_testing(img5))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


array([[0.41634133, 0.5836587 ]], dtype=float32)

In [ ]:
model.save("model.h5")

In [ ]:
os.listdir('/content/chest_xray/test')

['.DS_Store', 'PNEUMONIA', 'NORMAL']

In [ ]:
!rm -rf /content/chest_xray/test//.DS_Store

In [ ]:
os.listdir('/content/chest_xray/test')

['PNEUMONIA', 'NORMAL']

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

#load the model
model = load_model('model.h5')


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Your existing function to process individual images
def process_image_for_testing(my_img):
    img = image.load_img(my_img, target_size=(224, 224))  # Resize image to the target size
    img_tensor = image.img_to_array(img)  # Convert image to numpy array
    img_tensor = np.expand_dims(img_tensor, axis=0)  # Expand dimensions to match the model input
    img_tensor /= 255.  # Normalize pixel values to [0, 1]
    return img_tensor

# Function to make predictions for all images in a directory (NORMAL and PNEUMONIA)
def predict_for_test_directory(test_dir):
    predictions = []
    labels = []  # To store true labels if needed for evaluation

    # Iterate over subdirectories (NORMAL and PNEUMONIA)
    for subdir in os.listdir(test_dir):
        subdir_path = os.path.join(test_dir, subdir)

        if os.path.isdir(subdir_path):  # Only process subdirectories
            for filename in os.listdir(subdir_path):
                img_path = os.path.join(subdir_path, filename)

                try:
                    # Process each image and make a prediction
                    img_tensor = process_image_for_testing(img_path)
                    prediction = model.predict(img_tensor)  # Predict class probabilities
                    predicted_class = np.argmax(prediction, axis=1)  # Get the class with highest probability

                    # Store the prediction and the true label (0 for NORMAL, 1 for PNEUMONIA)
                    predictions.append(predicted_class[0])
                    labels.append(0 if subdir == 'NORMAL' else 1)  # Assign label based on subdirectory name

                except Exception as e:
                    print(f"Error processing {filename}: {e}")

    return np.array(predictions), np.array(labels)

# Assuming the model is already loaded (if not, load it)
# model = load_model('path_to_your_model.h5')  # Uncomment if the model is not already loaded

# Path to your test directory
test_dir = '/content/chest_xray/test'

# Get predictions and labels for all images in the test dataset
predictions, labels = predict_for_test_directory(test_dir)

# Print the predictions and true labels
print(predictions)  # Predicted class indices (0 for NORMAL, 1 for PNEUMONIA)
print(labels)  # True labels (0 for NORMAL, 1 for PNEUMONIA)

# Optionally, evaluate the model (if you have the true labels)
# from sklearn.metrics import accuracy_score
# print(f"Accuracy: {accuracy_score(labels, predictions)}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━